In [1]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import copy
import time

In [2]:
def open_data(fname):
    with open(fname, 'r') as f:
        m, p = list(map(int, f.readline().split()))

        data = np.zeros((m, p))

        for i in range(m):
            rowdata = list(map(lambda x: int(x) - 1, f.readline().split()))
            row, parts = rowdata[0], rowdata[1:]

            data[row, parts] = 1

    return m, p, data

In [3]:
def cluster_machines(data, parts_clusters, num_clusters):
    m, p = data.shape
    machine_clusters = [[] for i in range(num_clusters)]

    for i in range(m):
        curr_cluster = None
        current_cluster_ve = None
        for clu in range(num_clusters):
            v = int(np.sum(data[i, :]) - np.sum(data[i, parts_clusters[clu]] == 1))
            e = int(np.sum(data[i, parts_clusters[clu]] == 0))
            if current_cluster_ve == None or v + e < current_cluster_ve:
                current_cluster_ve = v + e
                curr_cluster = clu
        machine_clusters[curr_cluster].append(i)

    return machine_clusters

def f_obj(data, parts_clusters, machine_clusters):
    num_clusters = len(parts_clusters)

    n1 = np.sum(data)
    n1_in = 0
    n0_in = 0
    for i in range(num_clusters):
        n1_in += np.sum(data[machine_clusters[i], :][:, parts_clusters[i]] == 1)
        n0_in += np.sum(data[machine_clusters[i], :][:, parts_clusters[i]] == 0)

    return n1_in / (n1 + n0_in)

def single_move(i, cluster_from, cluster_to, data, parts_clusters):
    m, p = data.shape
    num_clusters = len(parts_clusters)

    pc = copy.deepcopy(parts_clusters)
    pc[cluster_from].remove(i)
    pc[cluster_to].append(i)

    mc = cluster_machines(data, pc, num_clusters)

    score = f_obj(data, pc, mc)

    return pc, mc, score

def find_cluster(elem, parts_clusters):
    for clu in range(len(parts_clusters)):
        if elem in parts_clusters[clu]:
            return clu

def best_single_move(data, parts_clusters):
    m, p = data.shape
    best_pc, best_mc, best_score = None, None, None
    for picked_elem in range(p):
        cluster_from = find_cluster(picked_elem, parts_clusters)
        for cluster_to in range(len(parts_clusters)):
            if cluster_to == cluster_from: 
                continue

            curr_pc, curr_mc, curr_score = single_move(picked_elem, cluster_from, cluster_to, data, parts_clusters)

            if best_score == None or curr_score > best_score:
                best_pc, best_mc, best_score = curr_pc, curr_mc, curr_score

    return best_pc, best_mc, best_score

def exchange_move(i, j, cluster_from, cluster_to, data, parts_clusters):
    m, p = data.shape
    num_clusters = len(parts_clusters)

    pc = copy.deepcopy(parts_clusters)
    pc[cluster_from].remove(i)
    pc[cluster_to].append(i)

    pc[cluster_to].remove(j)
    pc[cluster_from].append(j)

    mc = cluster_machines(data, pc, num_clusters)

    score = f_obj(data, pc, mc)

    return pc, mc, score

def best_exchange_move(data, parts_clusters):
    m, p = data.shape
    best_pc, best_mc, best_score = None, None, None
    for i in range(p):
        cluster_from = find_cluster(i, parts_clusters)
        for j in range(i+1, p):
            cluster_to = find_cluster(j, parts_clusters)
            if cluster_from == cluster_to:
                continue

            curr_pc, curr_mc, curr_score = exchange_move(i, j, cluster_from, cluster_to, data, parts_clusters)

            if best_score == None or curr_score > best_score:
                best_pc, best_mc, best_score = curr_pc, curr_mc, curr_score

    return best_pc, best_mc, best_score

In [4]:
def generate_initial_solution(num_clusters, data):
    m, p = data.shape
    sim_mat = np.eye(p)
    for i in range(p):
        for j in range(i + 1, p):
            a_ij = np.sum((data[:, i] == 1) & (data[:, j] == 1))
            b_ij = np.sum((data[:, i] == 1) & (data[:, j] == 0))
            c_ij = np.sum((data[:, i] == 0) & (data[:, j] == 1))

            sim_mat[i, j] =  a_ij / (a_ij + b_ij + c_ij)

    sim_mat += sim_mat.T - np.eye(p)

    clu = AgglomerativeClustering(n_clusters=num_clusters, metric='precomputed', linkage='average').fit_predict(sim_mat)
    parts_clusters = [[] for i in range(num_clusters)]
    for i in range(p):
        parts_clusters[clu[i]].append(i)

    machine_clusters = cluster_machines(data, parts_clusters, num_clusters)

    return parts_clusters, machine_clusters, f_obj(data, parts_clusters, machine_clusters)

In [5]:
def solve(data, T0, Tf, alpha, L, D, max_stagnant):
    C = 2
    S_best = None
    C_best = C

    while True:
        S = generate_initial_solution(C, data)
        S_cell = S
        if S_best == None:
            S_best = S_cell
        T = T0
        counter, counter_mc, counter_trapped, counter_stagnant = 0, 0, 0, 0
        while True:
            while counter_mc < L and counter_trapped < (L/2):
                if counter % D == 0:
                    Sc = best_exchange_move(data, S[0])
                else:
                    Sc = best_single_move(data, S[0])
                    
                if Sc[2] > S_cell[2]:
                    S_cell = Sc
                    S = Sc
                    counter_stagnant = 0
                    counter_mc += 1
                elif Sc[2] == S_cell[2]:
                    S = Sc
                    counter_stagnant += 1
                    counter_mc += 1
                else:
                    delta = Sc[2] - S[2]
                    x = np.random.uniform(0, 1)
                    proba = np.exp(delta/T) # если добавить минус то будут вероятности > 1. 
                                            # Минус уже предусмотрен, т.к всегда delta <= 0
                    assert proba < 1.0, 'Proba > 1'
                    if proba > x:
                        S = Sc
                        counter_trapped = 0
                    else:
                        counter_trapped += 1
                    counter_mc += 1
            if T <= Tf or counter_stagnant > max_stagnant:
                break
            else:
                T = T * alpha
                counter_mc = 0
                counter += 1
        if S_cell[2] > S_best[2]:
            S_best = S_cell
            C_best = C
            C += 1
        else:
            break
    return S_best

In [18]:
directory = 'cfp_data'
instance_name = '20x20'

fname = f'{directory}/{instance_name}.txt'
m, p, data = open_data(fname)

start = time.perf_counter()
pc, mc, final_score = solve(data, 
      T0=0.8,
      Tf=0.3,
      alpha=0.85,
      L=10,
      D=5,
      max_stagnant=5)
end = time.perf_counter()

print('Time:', end - start)
print('Solution score:', final_score)

with open(f'ans/{instance_name}.sol', 'w') as f:
    f.write(' '.join([f'{i + 1}_{find_cluster(i, mc) + 1}' for i in range(m)]))
    f.write('\n')
    f.write(' '.join([f'{i + 1}_{find_cluster(i, pc) + 1}' for i in range(p)]))

Time: 29.055456199999753
Solution score: 0.4230769230769231


In [12]:
directory = 'cfp_data'
instance_name = '24x40'


fname = f'{directory}/{instance_name}.txt'
m, p, data = open_data(fname)

start = time.perf_counter()
pc, mc, final_score = solve(data, 
      T0=0.8,
      Tf=0.3,
      alpha=0.85,
      L=10,
      D=5,
      max_stagnant=5)
end = time.perf_counter()

print('Time:', end - start)
print('Solution score:', final_score)

with open(f'ans/{instance_name}.sol', 'w') as f:
    f.write(' '.join([f'{i + 1}_{find_cluster(i, mc) + 1}' for i in range(m)]))
    f.write('\n')
    f.write(' '.join([f'{i + 1}_{find_cluster(i, pc) + 1}' for i in range(p)]))

Time: 573.8442511
Solution score: 0.45161290322580644


In [6]:
directory = 'cfp_data'
instance_name = '30x50'


fname = f'{directory}/{instance_name}.txt'
m, p, data = open_data(fname)

start = time.perf_counter()
pc, mc, final_score = solve(data, 
      T0=0.8,
      Tf=0.3,
      alpha=0.75,
      L=5,
      D=5,
      max_stagnant=5)
end = time.perf_counter()

print('Time:', end - start)
print('Solution score:', final_score)
 
with open(f'ans/{instance_name}.sol', 'w') as f:
    f.write(' '.join([f'{i + 1}_{find_cluster(i, mc) + 1}' for i in range(m)]))
    f.write('\n')
    f.write(' '.join([f'{i + 1}_{find_cluster(i, pc) + 1}' for i in range(p)]))

Time: 866.0935946000001
Solution score: 0.450261780104712


In [7]:
directory = 'cfp_data'
instance_name = '30x90'


fname = f'{directory}/{instance_name}.txt'
m, p, data = open_data(fname)

start = time.perf_counter()
pc, mc, final_score = solve(data, 
      T0=0.8,
      Tf=0.3,
      alpha=0.75,
      L=5,
      D=5,
      max_stagnant=5)
end = time.perf_counter()

print('Time:', end - start)
print('Solution score:', final_score)

with open(f'ans/{instance_name}.sol', 'w') as f:
    f.write(' '.join([f'{i + 1}_{find_cluster(i, mc) + 1}' for i in range(m)]))
    f.write('\n')
    f.write(' '.join([f'{i + 1}_{find_cluster(i, pc) + 1}' for i in range(p)]))

Time: 324.8137502999999
Solution score: 0.2934472934472934


In [8]:
directory = 'cfp_data'
instance_name = '37x53'


fname = f'{directory}/{instance_name}.txt'
m, p, data = open_data(fname)

start = time.perf_counter()
pc, mc, final_score = solve(data, 
      T0=0.8,
      Tf=0.3,
      alpha=0.75,
      L=5,
      D=5,
      max_stagnant=5)
end = time.perf_counter()

print('Time:', end - start)
print('Solution score:', final_score)

with open(f'ans/{instance_name}.sol', 'w') as f:
    f.write(' '.join([f'{i + 1}_{find_cluster(i, mc) + 1}' for i in range(m)]))
    f.write('\n')
    f.write(' '.join([f'{i + 1}_{find_cluster(i, pc) + 1}' for i in range(p)]))

Time: 25.436568999999963
Solution score: 0.6005314437555359
